In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

#爬取过程中一直报LineTooLong: got more than 65536 bytes when reading header line的错误，需把字节设置大解决
import http.client
http.client._MAXLINE = 524288 

1. 在猎聘网中，输入职位关键词：python，找到网页中的数据来源。抓取这个职位一定数量的招聘信息，抓取的信息可以包括薪水、学历要求、工作经验要求、招聘详细页url、公司名、公司 url 等；

In [2]:
headers={
    "Cookie": "abtest=0; __uuid=1588492998549.23; __tlog=1588492998552.56%7C00000000%7C00000000%7C00000000%7C00000000; __s_bid=c46867385fbb56ec4c7db53d11e13e013b2b; Hm_lvt_a2647413544f5a04f00da7eee0d5e200=1588492999; fe_se=-1588493944090; JSESSIONID=C23ED61672057E24E9B7E779AF29530F; __session_seq=3; __uv_seq=3; Hm_lpvt_a2647413544f5a04f00da7eee0d5e200=1588494020",
    "Host": "www.liepin.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36"
}
lists=[]

In [3]:
#job的url规则不一样，要分情况拼写
def url_check(url):
    if url[0]=='/':
        return 'https://www.liepin.com'+url
    else:
        return url

In [4]:
#抓取该职位的详细职位描述
def job_detailed(url):
    req1=requests.get(url,headers=headers)
    req1.encoding='utf-8'
    soup1=BeautifulSoup(req1.content,'html.parser')
    detailed=soup1.select('.content-word')[0].get_text(strip=True)
    return detailed
job_detailed('https://www.liepin.com/job/1925852925.shtml')

'职责描述：1、从事机器学习技术的应用开发；2、负责公司项目中的图像识别模块的研发，负责图像处理、机器视觉的算法的实现；3、通过集成图像处理、机器视觉的算法实现具有特定功能的图像识别系统；4、结合大数据和互联网产品的需求，对外提供图像识别技术服务；任职要求：1、全日制一类本科学历以上，高考录取分数线在560分以上，年龄23-28，数学或者计算机专业，3年以上工作经验；2、熟悉python及相关的库，熟悉linux环境，具备良好的代码书写规范和文档编写能力；3、熟悉常用数据结构与算法，掌握Redis、MongoDB、Mysql等常用数据库；4、熟练使用 Git，熟悉 Pull Request；5、掌握计算机视觉和图像处理基本算法，对图像分割、特征提取、分类器识别有较深入研究；对于图像识别的相关算法能够用计算机语言进行很好的实现；6、有计算机视觉、图像处理、机器学习，卷积神经网络应用等方面的项目经历的优先；7、具有强烈的质量意识，工作认真、细致、有耐心，有良好的团队协作意识 ，乐于接受挑战，思路清晰，善于钻研思考；8、薪资待遇：试用期8000-15000元，具体根据个人工作能力面议。'

In [6]:
for i in range(5):
    url='https://www.liepin.com/zhaopin/?init=-1&headckid=cca369b4408b04b5&fromSearchBtn=2&dqs=170020&ckid=b28bc90da5cde634&degradeFlag=0\
    &sfrom=click-pc_homepage-centre_searchbox-search_new&key=python&siTag=I-7rQ0e90mv8a37po7dV3Q%7EILIFxAzMuTzzOU47n1bPhg&d_sfrom=search_fp\
    &d_ckId=9a3ed3bb3fb3f7d3df6dad1462b734bd&d_curPage=0&d_pageSize=40&d_headId=59d2185408578ddd56090a65bbedfd3b&curPage={}'.format(i)
    req=requests.get(url,headers=headers)
    req.encoding='utf-8'
    soup=BeautifulSoup(req.content,'html.parser')
    sojobs=soup.find_all(class_='sojob-item-main clearfix')
    for sojob in sojobs:
        job_post=sojob.select('h3 a')[0].get_text(strip=True)
        job_url=sojob.select('div.job-info a')[0]['href'].strip()
        job_url=url_check(job_url)
        jobs=[j.get_text(strip=True) for j in sojob.select('div.job-info p span')]
        job_salary=jobs[0]
        job_education=jobs[1]
        job_exp=jobs[2]
        
        com_name=sojob.select('.company-info.nohover .company-name a')[0].get_text(strip=True)
        try:
            com_url=sojob.select('.company-info.nohover .company-name a')[0]['href']
        except:
            com_url=sojob.select('.company-info.nohover .honesty-tips')[0].get_text(strip=True)
        ls=[job_post,job_url,job_salary,job_education,job_exp,com_name,com_url,job_detailed(job_url)]
        lists.append(ls)
    time.sleep(5)
df=pd.DataFrame(lists,columns=['招聘岗位','招聘详细页url','薪水','学历要求','工作经验要求','公司名','公司url','详细职位描述'])
df.head()

,招聘岗位,招聘详细页url,薪水,学历要求,工作经验要求,公司名,公司url,详细职位描述
0,Python开发工程师,https://www.liepin.com/job/1927986643.shtml,20-45k·12薪,统招本科,经验不限,卓尔人人,https://www.liepin.com/company/12146335/,职责描述：1. 负责公司项目的设计和开发；2. 持续改进现有代码以应对多样化需求；3. 维护...
1,高学历人才储备（Python开发经理）,https://www.liepin.com/job/1925852925.shtml,8-15k·12薪,本科及以上,3-5年,恒信汽车集团股份有限公司,https://www.liepin.com/company/8637943/,职责描述：1、从事机器学习技术的应用开发；2、负责公司项目中的图像识别模块的研发，负责图像处...
2,Python开发工程师,https://www.liepin.com/job/1927917319.shtml,10-15k·12薪,硕士及以上,经验不限,武大吉奥,https://www.liepin.com/company/1230425/,岗位职责1、构建数据的智能化处理框架，利用数理统计、数据挖掘、自然语言处理等多种技术手段对海...
3,大数据开发工程师（python方向）,https://www.liepin.com/job/1924369011.shtml,10-15k·17薪,本科及以上,5-10年,武汉中海庭数据技术有限公司,https://www.liepin.com/company/9644059/,任职要求：1、统招本科以上学历，计算机相关专业，3年以上实际Python项目开发实施工作经验...
4,Python开发工程师,https://www.liepin.com/job/1927895377.shtml,8-18k·12薪,本科及以上,1-3年,武汉康圣达医学检验所有限公司,https://www.liepin.com/company/9283653/,岗位职责：1、负责基于python和CS架构的软件设计及实现；2、负责基于QT的软件界面及功...


In [9]:
df.to_csv('猎聘网python职位.csv',index=0)